## Deliverable 2. Create a Customer Travel Destinations Map.

In [7]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [14]:
# 1. Import the WeatherPy_database.csv file.
city_data_df = pd.read_csv ("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Current Description,Date
0,0,Kapaa,22.0752,-159.3190,73.72,77,100,4.61,US,overcast clouds,2022-02-12 07:21:44
1,1,Hobyo,5.3505,48.5268,80.76,53,97,17.92,SO,overcast clouds,2022-02-12 07:21:45
2,2,Kodiak,57.7900,-152.4072,38.71,75,0,3.44,US,clear sky,2022-02-12 07:21:46
3,3,Ushuaia,-54.8000,-68.3000,51.46,62,40,51.79,AR,scattered clouds,2022-02-12 07:21:46
4,4,Bredasdorp,-34.5322,20.0403,75.27,68,88,12.66,ZA,overcast clouds,2022-02-12 07:21:47


In [15]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 79
What is the maximum temperature you would like for your trip? 90


In [16]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)
# preferred_cities_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Current Description,Date
1,1,Hobyo,5.3505,48.5268,80.76,53,97,17.92,SO,overcast clouds,2022-02-12 07:21:45
22,22,Hithadhoo,-0.6000,73.0833,81.27,69,40,5.28,MV,scattered clouds,2022-02-12 07:21:15
27,27,Trat,12.5000,102.5000,89.26,52,66,6.38,TH,broken clouds,2022-02-12 07:21:57
32,32,Cairns,-16.9167,145.7667,88.07,54,4,12.66,AU,clear sky,2022-02-12 07:21:59
33,33,Isangel,-19.5500,169.2667,81.25,83,99,9.71,VU,overcast clouds,2022-02-12 07:21:59
45,45,Pathein,16.7833,94.7333,87.71,26,40,17.02,MM,scattered clouds,2022-02-12 07:23:04
48,48,Rikitea,-23.1203,-134.9692,79.27,73,6,18.30,PF,light rain,2022-02-12 07:23:05
52,52,Butaritari,3.0707,172.7902,82.74,73,98,15.70,KI,overcast clouds,2022-02-12 07:23:07
53,53,Souillac,-20.5167,57.5167,84.36,78,40,13.80,MU,scattered clouds,2022-02-12 07:23:08
54,54,Pontianak,-0.0333,109.3333,87.78,66,75,6.91,ID,broken clouds,2022-02-12 07:23:08


In [18]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()


City_ID                119
City                   119
Lat                    119
Lng                    119
Max Temp               119
Humidity               119
Cloudiness             119
Wind Speed             119
Country                118
Current Description    119
Date                   119
dtype: int64

In [19]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.

# if any, use dropna()method at the end of your filtering statement when you are creating the new DataFrame


In [21]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Hobyo,SO,80.76,overcast clouds,5.3505,48.5268,
22,Hithadhoo,MV,81.27,scattered clouds,-0.6000,73.0833,
27,Trat,TH,89.26,broken clouds,12.5000,102.5000,
32,Cairns,AU,88.07,clear sky,-16.9167,145.7667,
33,Isangel,VU,81.25,overcast clouds,-19.5500,169.2667,
45,Pathein,MM,87.71,scattered clouds,16.7833,94.7333,
48,Rikitea,PF,79.27,light rain,-23.1203,-134.9692,
52,Butaritari,KI,82.74,overcast clouds,3.0707,172.7902,
53,Souillac,MU,84.36,scattered clouds,-20.5167,57.5167,
54,Pontianak,ID,87.78,broken clouds,-0.0333,109.3333,


In [24]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("hotel isn't found skip the city.")
        

hotel isn't found skip the city.
hotel isn't found skip the city.
hotel isn't found skip the city.
hotel isn't found skip the city.
hotel isn't found skip the city.
hotel isn't found skip the city.
hotel isn't found skip the city.
hotel isn't found skip the city.
hotel isn't found skip the city.
hotel isn't found skip the city.
hotel isn't found skip the city.
hotel isn't found skip the city.
hotel isn't found skip the city.
hotel isn't found skip the city.


In [25]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.head(10)
#preferred_cities_df.count()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Hobyo,SO,80.76,overcast clouds,5.3505,48.5268,Halane Hotel and Restaurent
22,Hithadhoo,MV,81.27,scattered clouds,-0.6000,73.0833,Scoop Guest House
27,Trat,TH,89.26,broken clouds,12.5000,102.5000,สวนหนิง-อัศ
32,Cairns,AU,88.07,clear sky,-16.9167,145.7667,Mantra Trilogy Cairns
33,Isangel,VU,81.25,overcast clouds,-19.5500,169.2667,Tanna Lodge
45,Pathein,MM,87.71,scattered clouds,16.7833,94.7333,New Pammawaddy Hotel
48,Rikitea,PF,79.27,light rain,-23.1203,-134.9692,People ThankYou
52,Butaritari,KI,82.74,overcast clouds,3.0707,172.7902,Isles Sunset Lodge
53,Souillac,MU,84.36,scattered clouds,-20.5167,57.5167,Shanti Maurice Resort & Spa
54,Pontianak,ID,87.78,broken clouds,-0.0333,109.3333,Aston Pontianak Hotel & Convention Center


In [31]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [44]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [45]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))